In [1]:
!pip install rasterio

In [4]:
# =========================
# INSTALLATION & IMPORTS
# =========================

import os
import ee
import glob
import time
import geemap
import pprint
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm import tqdm
from datetime import datetime, timedelta
from rasterio.features import rasterize
from rasterio.transform import from_bounds

import torch
import torch.nn as nn

import google.auth
from google.auth import compute_engine
from google.oauth2 import service_account

# Service account JSON file
service_account_key = '/content/drive/MyDrive/AGRI/jsonKey/ee-chriscandido93-d6ab0900647b.json'
credentials = service_account.Credentials.from_service_account_file(
    service_account_key,
    scopes=['https://www.googleapis.com/auth/earthengine']
)


#credentials = compute_engine.Credentials(scopes=scopes)
ee.Initialize(credentials, project='ee-chriscandido93')

In [12]:
# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
  print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cpu


In [5]:
gpkg_dir = '/content/drive/MyDrive/AGRI/Planting_Method/tiles'

bounds_list = []

for file in os.listdir(gpkg_dir):
    if file.endswith('.gpkg'):
        file_path = os.path.join(gpkg_dir, file)
        gdf = gpd.read_file(file_path)

        min_lon, min_lat, max_lon, max_lat = gdf.total_bounds
        bounds_list.append({
            "filename": file,
            "min_lon": min_lon,
            "min_lat": min_lat,
            "max_lon": max_lon,
            "max_lat": max_lat
        })

# convert to DataFrame
bounds_df = pd.DataFrame(bounds_list)
print(bounds_df)

                 filename     min_lon    min_lat     max_lon    max_lat
0        tile_laguna.gpkg  120.984459  13.958397  121.638203  14.610176
1    tile_nuevaecija.gpkg  120.587638  15.171764  121.378589  16.135426
2   tile_philippines.gpkg  119.910107   6.245471  127.062292  18.676649
3  tile_nuevavizcaya.gpkg  120.757067  15.766210  121.573283  16.766733
4       tile_Isabela.gpkg  121.345094  16.367402  122.433382  17.609279


In [6]:
area = 'tile_nuevavizcaya.gpkg'

min_lon = bounds_df[bounds_df['filename'] == area]['min_lon']
min_lat = bounds_df[bounds_df['filename'] == area]['min_lat']
max_lon = bounds_df[bounds_df['filename'] == area]['max_lon']
max_lat = bounds_df[bounds_df['filename'] == area]['max_lat']

min_lon = min_lon.values[0]
min_lat = min_lat.values[0]
max_lon = max_lon.values[0]
max_lat = max_lat.values[0]

In [7]:
# =========================
# 1. DEFINE AOI AND TIME RANGE
# =========================

# min_lon, min_lat, max_lon, max_lat = 119.735, 12.516, 121.735, 15.516
aoi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

start_date = '2024-10-01'
end_date = '2025-06-30'

print(f"\nAOI Bounds: ({min_lon}, {min_lat}) to ({max_lon}, {max_lat})")
print(f"Time Range: {start_date} to {end_date}\n")


AOI Bounds: (120.75706731102576, 15.766210320409542) to (121.57328347795568, 16.766733363743004)
Time Range: 2024-10-01 to 2025-06-30



In [8]:
# =========================
# 2. CREATE TILES WITH OVERLAP
# =========================

def create_tiles_with_overlap(min_lon, min_lat, max_lon, max_lat, tile_size, overlap=0.1):
    """
    Create tiles with overlap between adjacent tiles

    Parameters:
    - min_lon, min_lat, max_lon, max_lat: Bounding box coordinates
    - tile_size: Size of each tile in degrees
    - overlap: Overlap percentage between tiles (0.1 = 10% overlap)
    """
    tiles = []
    overlap_distance = tile_size * overlap

    lon = min_lon
    while lon < max_lon:
        lat = min_lat
        while lat < max_lat:
            tile_min_lon = lon
            tile_min_lat = lat
            tile_max_lon = min(lon + tile_size + overlap_distance, max_lon)
            tile_max_lat = min(lat + tile_size + overlap_distance, max_lat)

            tile = ee.Geometry.Rectangle([
                tile_min_lon,
                tile_min_lat,
                tile_max_lon,
                tile_max_lat
            ])
            tiles.append(tile)
            lat += tile_size
        lon += tile_size

    print(f"Generated {len(tiles)} tiles with {overlap*100}% overlap")
    return tiles

# Create tiles with 10% overlap
tiles = create_tiles_with_overlap(min_lon, min_lat, max_lon, max_lat, 0.5, overlap=0.1)

Generated 6 tiles with 10.0% overlap


In [9]:
polarization = 'VH'

def smoothing(image):

  return image.focalMode(**{'radius':1, 'kernelType':'circle', 'units':'pixels', 'iterations':10})

def mask_edge(image):
    """Mask out edges with very low backscatter"""
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)


def refined_lee_filter(image, polarization='VH', kernel_size=3):
    """
    Apply Refined Lee filter to reduce speckle noise
    FIXED: Properly handles array operations and returns regular image

    Parameters:
        image: ee.Image with polarization band
        polarization: str, polarization type ('VV' or 'VH')
        kernel_size: int, kernel size (default 3)

    Returns:
        ee.Image with filtered band
    """
    # Select the polarization band
    img_band = image.select(polarization)

    # Convert from dB to linear scale
    img_linear = ee.Image(10).pow(img_band.divide(10))

    # Define kernel
    kernel = ee.Kernel.square(kernel_size / 2, 'pixels', False)

    # Calculate local mean
    mean = img_linear.reduceNeighborhood(
        reducer=ee.Reducer.mean(),
        kernel=kernel
    )

    # Calculate local variance
    variance = img_linear.reduceNeighborhood(
        reducer=ee.Reducer.variance(),
        kernel=kernel
    )

    # Estimate noise variance
    # Use a simpler approach that doesn't create array issues
    sample_weights = variance.divide(mean.pow(2))

    # Create a larger kernel for noise estimation
    noise_kernel = ee.Kernel.square(kernel_size * 2, 'pixels', False)

    # Get noise variance estimate (using minimum in neighborhood)
    noise_variance = sample_weights.reduceNeighborhood(
        reducer=ee.Reducer.min(),
        kernel=noise_kernel
    )

    # Refined Lee formula
    # varX = (variance - mean^2 * noise_variance) / (noise_variance + 1)
    varX = variance.subtract(
        mean.pow(2).multiply(noise_variance)
    ).divide(
        noise_variance.add(1.0)
    )

    # Weighting factor
    b = varX.divide(variance)

    # Apply filter
    filtered = mean.add(
        b.multiply(img_linear.subtract(mean))
    )

    # Convert back to dB
    filtered_db = filtered.log10().multiply(10)

    # Get date for naming
    date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    custom_name = date_str.cat(f'_S1{polarization}')

    # Return as regular image (not array)
    return (filtered_db
            .rename([polarization])
            .set('name', custom_name)
            .set('key', custom_name)
            .copyProperties(image, ['system:time_start', 'system:id']))

# Fix empty mosaics to avoid 0-band images
def fix_empty(img, band_name):
    img = ee.Image(img)  # if img is NULL → returns a NULL placeholder
    return ee.Image(
        ee.Algorithms.If(
            img,
            ee.Algorithms.If(
                img.bandNames().size().eq(0),
                ee.Image(0).rename(band_name).updateMask(ee.Image(1)),
                img
            ),
            ee.Image(0).rename(band_name).updateMask(ee.Image(1))
        )
    )

def temporal_linear_interpolation(ic):

    def interp(img):
        img = ee.Image(img)
        t = img.date().millis()

        # Get previous image
        prev = ic.filterDate(
            ee.Date(t).advance(-500, 'day'),
            ee.Date(t)
        ).sort('system:time_start', False).first()

        # Get next image
        next_ = ic.filterDate(
            ee.Date(t),
            ee.Date(t).advance(500, 'day')
        ).sort('system:time_start').first()

        # Ensure prev/next aren't empty
        prev = fix_empty(prev, polarization)
        next_ = fix_empty(next_, polarization)
        img  = fix_empty(img, polarization)

        prev = ee.Image(prev)
        next_ = ee.Image(next_)

        prev_t = ee.Number(prev.get('system:time_start'))
        next_t = ee.Number(next_.get('system:time_start'))

        # Avoid divide-by-zero in rare cases
        ratio = ee.Number(t).subtract(prev_t).divide(
            next_t.subtract(prev_t).max(1)
        )

        # Linear interpolation: prev + ratio*(next - prev)
        interpolated = prev.add(
            next_.subtract(prev).multiply(ratio)
        )

        # Fill missing pixels in original with interpolated values
        filled = img.unmask(interpolated)

        return filled.copyProperties(img).set('system:time_start', t)

    return ic.map(interp)

In [10]:
collectionS1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
                .filter(ee.Filter.eq('instrumentMode', 'IW'))
                .filterDate(start_date, end_date)
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))
                .filterBounds(aoi)
                .map(mask_edge))

# Filter by descending orbit properties only
collectionS1_desc = collectionS1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

# Select VH band and apply a refined Lee filter
collectionS1_desc_VH = collectionS1_desc.select(['VH']).map(refined_lee_filter)

# Function to sample images at biweekly intervals (14 days)
def sample_images_at_intervals(collection, start_date, end_date, interval_days):
    date_list = ee.List.sequence(ee.Date(start_date).millis(), ee.Date(end_date).millis(), interval_days * 24 * 60 * 60 * 1000)

    def filter_and_get_image(millis):
        filtered = collection.filterDate(ee.Date(millis), ee.Date(millis).advance(interval_days, 'day')).mosaic()
        renamed_image = filtered.rename(polarization)
        image_date = ee.Date(millis).format('YYYY-MM-dd')
        image_name = ee.String('S1_Mosaic_').cat(image_date)
        return renamed_image.set('system:time_start', millis, 'system:id', image_name)

    sampled_images = date_list.map(filter_and_get_image)
    return ee.ImageCollection.fromImages(sampled_images)

# Sample images at BIWEEKLY intervals (changed from 1 to 14)
biweekly_images_desc_mosaicked = sample_images_at_intervals(collectionS1_desc_VH, start_date, end_date, 14)

# Sort the biweekly mosaicked images by date
sorted_combined_images = biweekly_images_desc_mosaicked.sort('system:time_start')

In [11]:
collectionS1

In [ ]:

def create_linear_interpolated_timeseries(aoi, start_date, end_date, polarization='VH',
                                         interval_days=14, expected_count=18):
    """
    Enhanced linear interpolation with better missing value handling.
    Interpolates temporally between available images and fills gaps.
    """

    # Collect S1 data
    collectionS1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
                    .filter(ee.Filter.eq('instrumentMode', 'IW'))
                    .filterDate(start_date, end_date)
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))
                    .filterBounds(aoi)
                    .map(mask_edge))

    collectionS1_desc = collectionS1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    collectionS1_desc_pol = collectionS1_desc.select([polarization]).map(refined_lee_filter)

    # Generate target dates
    date_list = ee.List.sequence(
        ee.Date(start_date).millis(),
        ee.Date(end_date).millis(),
        interval_days * 24 * 60 * 60 * 1000
    ).slice(0, expected_count)

    # For each target date, find closest images and interpolate
    def interpolate_for_date(millis):
        target_date = ee.Date(millis)

        # Extended search window for better interpolation
        search_window = interval_days * 2  # Look further for interpolation candidates
        window_start = target_date.advance(-search_window, 'day')
        window_end = target_date.advance(search_window, 'day')

        nearby = collectionS1_desc_pol.filterDate(window_start, window_end)

        # Check for exact match in smaller window
        exact_window = interval_days / 2
        exact_match = collectionS1_desc_pol.filterDate(
            target_date.advance(-exact_window, 'day'),
            target_date.advance(exact_window, 'day')
        )

        has_exact = exact_match.size().gt(0)

        # If exact match exists, use it
        def use_exact():
            return exact_match.mosaic()

        # Otherwise, interpolate from nearby images
        def interpolate_nearby():
            # Get images before and after target date
            before = nearby.filterDate(window_start, target_date).sort('system:time_start', False)
            after = nearby.filterDate(target_date, window_end).sort('system:time_start')

            has_before = before.size().gt(0)
            has_after = after.size().gt(0)

            # Linear interpolation between closest before and after images
            def linear_interp():
                img_before = ee.Image(before.first())
                img_after = ee.Image(after.first())

                date_before = ee.Date(img_before.get('system:time_start'))
                date_after = ee.Date(img_after.get('system:time_start'))

                # Calculate weights based on temporal distance
                total_diff = date_after.difference(date_before, 'day')
                weight_after = target_date.difference(date_before, 'day').divide(total_diff)
                weight_before = ee.Number(1).subtract(weight_after)

                # Weighted average for smooth interpolation
                interpolated = img_before.multiply(weight_before).add(
                    img_after.multiply(weight_after)
                )

                return interpolated

            # Fallback: use nearest available image
            def use_before():
                return before.first()

            def use_after():
                return after.first()

            # Last resort: use median of all available data
            def use_fallback():
                all_images = collectionS1_desc_pol
                return ee.Algorithms.If(
                    all_images.size().gt(0),
                    all_images.median(),
                    ee.Image.constant(0).rename(polarization)  # Fill with zeros if no data
                )

            # Decision tree for interpolation strategy
            return ee.Image(ee.Algorithms.If(
                has_before.And(has_after),
                linear_interp(),
                ee.Algorithms.If(
                    has_before,
                    use_before(),
                    ee.Algorithms.If(
                        has_after,
                        use_after(),
                        use_fallback()
                    )
                )
            ))

        result = ee.Image(ee.Algorithms.If(has_exact, use_exact(), interpolate_nearby()))
        return result.set('system:time_start', millis).set('interpolated', ee.Number(has_exact).Not())

    # Create interpolated collection
    interpolated_images = date_list.map(interpolate_for_date)
    interpolated_collection = ee.ImageCollection.fromImages(interpolated_images)

    # Stack into multi-band image
    band_list = ee.List.sequence(0, expected_count - 1)

    def rename_band(idx):
        idx = ee.Number(idx)
        img = ee.Image(interpolated_images.get(idx))
        band_name = idx.format('%d').cat('_').cat(polarization)
        return img.select([polarization]).rename(band_name)

    renamed_images = band_list.map(rename_band)

    # Combine all bands
    stacked = ee.ImageCollection.fromImages(renamed_images).toBands()

    # Remove the collection prefix from band names
    old_names = stacked.bandNames()

    def create_band_name(i):
        return ee.Number(i).format('%d').cat('_').cat(polarization)

    new_names = band_list.map(create_band_name)

    final_stack = stacked.rename(new_names)

    # Optional: Apply gap-filling for any remaining missing pixels
    final_stack = fill_remaining_gaps(final_stack, expected_count, polarization)

    return final_stack


def fill_remaining_gaps(stacked_image, band_count, polarization):
    """
    Fill any remaining gaps using linear interpolation across the time series.
    This handles pixel-level missing values within bands.
    """
    def create_band_name(i):
        return ee.Number(i).format('%d').cat('_').cat(polarization)

    band_names = ee.List.sequence(0, band_count - 1).map(create_band_name)

    def interpolate_band(band_idx):
        band_idx = ee.Number(band_idx)
        band_name = band_idx.format('%d').cat('_').cat(polarization)
        current_band = stacked_image.select([band_name])

        # Find previous and next valid bands for interpolation
        prev_idx = band_idx.subtract(1)
        next_idx = band_idx.add(1)

        has_prev = prev_idx.gte(0)
        has_next = next_idx.lt(band_count)

        def interp_both():
            prev_name = prev_idx.format('%d').cat('_').cat(polarization)
            next_name = next_idx.format('%d').cat('_').cat(polarization)
            prev_band = stacked_image.select([prev_name])
            next_band = stacked_image.select([next_name])
            return prev_band.add(next_band).divide(2)

        def use_prev():
            prev_name = prev_idx.format('%d').cat('_').cat(polarization)
            return stacked_image.select([prev_name])

        def use_next():
            next_name = next_idx.format('%d').cat('_').cat(polarization)
            return stacked_image.select([next_name])

        interpolated = ee.Image(ee.Algorithms.If(
            has_prev.And(has_next),
            interp_both(),
            ee.Algorithms.If(has_prev, use_prev(),
                ee.Algorithms.If(has_next, use_next(), current_band))
        ))

        # Use interpolated values only where original is masked
        return current_band.unmask(interpolated).rename([band_name])

    # Apply interpolation to all bands
    band_indices = ee.List.sequence(0, band_count - 1)
    interpolated_bands = band_indices.map(interpolate_band)

    return ee.ImageCollection.fromImages(interpolated_bands).toBands().rename(band_names)

In [ ]:
composite_interpolated = create_linear_interpolated_timeseries(
    aoi, start_date, end_date,
    polarization=polarization,
    interval_days=14,
    expected_count=18
)

In [ ]:
# Load ESA WorldCover for 2020 and select the cropland class (class 40)
esa_worldcover = ee.Image("ESA/WorldCover/v100/2020").select('Map')

# Create a cropland mask (1 for cropland, 0 for non-cropland)
cropland_mask = esa_worldcover.eq(40)

composite_mask = composite_interpolated.updateMask(cropland_mask)
composite_mask

In [ ]:
from ee.ee_list import List
# Basic validation - just check if bands exist and have non-zero data
print("Basic band validation...")

for i, band_name in enumerate(composite_mask.bandNames().getInfo()):
    band = composite_interpolated.select([band_name])

    # Quick check - get mean value
    mean_val = band.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=100,
        bestEffort=True
    ).getInfo().get(band_name)

    if mean_val is None:
        print(f"❌ {band_name}: No data")
    elif mean_val == 0:
        print(f"⚠️  {band_name}: Mean = 0 (possible no data)")
    else:
        print(f"✅ {band_name}: Mean = {mean_val:.6f}")

print(f"\n🎯 Your single image has {len(composite_mask.bandNames().getInfo())} bands ready for CSV extraction!")

Basic band validation...
✅ 0_VH: Mean = -14.215215
✅ 1_VH: Mean = -13.964316
✅ 2_VH: Mean = -13.900201
✅ 3_VH: Mean = -13.973609
✅ 4_VH: Mean = -14.009997
✅ 5_VH: Mean = -13.813339
✅ 6_VH: Mean = -13.923376
✅ 7_VH: Mean = -13.992149
✅ 8_VH: Mean = -14.087527
✅ 9_VH: Mean = -13.998667
✅ 10_VH: Mean = -14.159896
✅ 11_VH: Mean = -13.781698
✅ 12_VH: Mean = -14.393056
✅ 13_VH: Mean = -14.426889
✅ 14_VH: Mean = -14.659447
✅ 15_VH: Mean = -14.767458
✅ 16_VH: Mean = -14.674008
✅ 17_VH: Mean = -13.715917

🎯 Your single image has 18 bands ready for CSV extraction!


In [ ]:
def visualize_timeseries(time_series, aoi, polarization='VH'):
    """
    Create map visualization

    Args:
        time_series: ee.ImageCollection
        aoi: Area of interest
        polarization: 'VV' or 'VH'

    Returns:
        geemap.Map
    """
    import geemap

    Map = geemap.Map()
    Map.centerObject(aoi, 12)

    # Add AOI
    Map.addLayer(ee.Image().paint(aoi, 0, 2), {'palette': 'yellow'}, 'AOI')

    # Visualization parameters
    vis = {
        'bands': ['0_VH', '5_VH', '10_VH'],
        'min': -30,
        'max': -5
    }


    start_date = time_series.get('start_date').getInfo()
    end_date = time_series.get('end_date').getInfo()


    # Create descriptive layer name
    layer_name = f'{start_date} to {end_date} - {polarization}'
    Map.addLayer(time_series, vis, layer_name, False)

    # Add latest image as visible by default
    latest_start = time_series.get('start_date').getInfo()
    latest_end = time_series.get('end_date').getInfo()
    latest_period = time_series.get('period').getInfo()

    latest_layer_name = f'★ Latest ({latest_period}): {latest_start} to {latest_end}'
    Map.addLayer(time_series, vis, latest_layer_name, True)


    return Map

In [ ]:
# Date range for biweekly composites

print(f"Creating visualization for first tile...")
example_tile = tiles[0]
aoi = ee.Geometry.Rectangle([113.6804, 3.1114, 129.9438, 22.2536])
Map = visualize_timeseries(composite_mask, aoi, 'VH')
Map

Creating visualization for first tile...


Map(center=[12.679436406891591, 121.81210000000002], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# =========================
# TFRECORD EXPORT FUNCTIONS
# =========================

def export_composite_as_tfrecord(composite, tiles, output_folder, prefix='sentinel1',
                                  patch_size=512, scale=10):
    """
    Export composite to TFRecord format with tiling to avoid memory limits

    Args:
        composite: ee.Image - The composite image to export
        tiles: list - List of tile geometries
        output_folder: str - Google Drive folder name for output
        prefix: str - Prefix for output files
        patch_size: int - Size of each patch in pixels (default: 256)
        scale: int - Export scale in meters (default: 10m for S2)

    Returns:
        list: Export tasks
    """

    print(f"\n{'='*70}")
    print("EXPORTING COMPOSITE AS TFRECORD")
    print(f"{'='*70}")
    print(f"Number of tiles: {len(tiles)}")
    print(f"Patch size: {patch_size}x{patch_size} pixels")
    print(f"Scale: {scale}m")
    print(f"Output folder: {output_folder}")

    # Get band names
    bands = composite.bandNames().getInfo()
    print(f"Bands: {bands}")

    # Prepare composite for export
    composite_export = composite.select(bands).float()

    # Create export tasks for each tile
    tasks = []

    for i, tile in enumerate(tiles):
        print(f"\n[Tile {i+1}/{len(tiles)}] Preparing export...")

        # Get tile bounds
        coords = tile.bounds().coordinates().getInfo()[0]
        tile_bounds = tile

        # Create export description
        description = f'{prefix}_tile_{i+1:03d}'

        # Configure export parameters
        export_params = {
            'image': composite_export.clip(tile_bounds),
            'description': description,
            'folder': output_folder,
            'fileNamePrefix': description,
            'scale': scale,
            'region': tile_bounds,
            'fileFormat': 'TFRecord',
            'maxPixels': 1e13,
            'formatOptions': {
                'patchDimensions': [patch_size, patch_size],
                'compressed': True,
                'maxFileSize': 104857600  # 100MB per file
            }
        }

        # Create export task
        task = ee.batch.Export.image.toDrive(**export_params)

        # Start the task
        task.start()
        tasks.append(task)

        print(f"   ✓ Task started: {description}")
        print(f"   Status: {task.status()['state']}")

    print(f"\n{'='*70}")
    print(f"✓ All {len(tasks)} export tasks started successfully!")
    print(f"{'='*70}")
    print("\nMONITORING:")
    print("  - Check task status in the next cell")
    print("  - Files will be saved to Google Drive")
    print(f"  - Folder: {output_folder}")
    print("\nNOTE:")
    print("  - Large exports may take several hours")
    print("  - You can close this notebook - tasks run on GEE servers")
    print("  - Monitor progress at: https://code.earthengine.google.com/tasks")

    return tasks


def monitor_export_tasks(tasks):
    """
    Monitor the status of export tasks

    Args:
        tasks: list - List of ee.batch.Task objects
    """
    import time

    print(f"\n{'='*70}")
    print("MONITORING EXPORT TASKS")
    print(f"{'='*70}\n")

    while True:
        states = {}
        for task in tasks:
            state = task.status()['state']
            states[state] = states.get(state, 0) + 1

        print(f"[{datetime.now().strftime('%H:%M:%S')}] Task Status:")
        for state, count in states.items():
            print(f"  {state}: {count}")

        # Check if all tasks are completed or failed
        if all(task.status()['state'] in ['COMPLETED', 'FAILED', 'CANCELLED']
               for task in tasks):
            break

        print("\nRefreshing in 60 seconds...")
        time.sleep(60)
        print("\n" + "-"*70)

    print(f"\n{'='*70}")
    print("✓ ALL TASKS FINISHED")
    print(f"{'='*70}\n")

    # Print final summary
    final_states = {}
    for task in tasks:
        state = task.status()['state']
        final_states[state] = final_states.get(state, 0) + 1

    print("FINAL SUMMARY:")
    for state, count in final_states.items():
        emoji = "✓" if state == "COMPLETED" else "✗"
        print(f"  {emoji} {state}: {count}")

In [ ]:
# Configure export parameters
OUTPUT_FOLDER = 'tfrecord'  # Google Drive folder name
PREFIX = 'S1_composite_dry2025'
PATCH_SIZE = 256  # 256x256 pixel patches
SCALE = 10  # 10m resolution for Sentinel-2

# Start export
export_tasks = export_composite_as_tfrecord(
    composite=composite_mask,
    tiles=tiles,
    output_folder=OUTPUT_FOLDER,
    prefix=PREFIX,
    patch_size=PATCH_SIZE,
    scale=SCALE
)

print(f"\n{'='*70}")
print(f"Total tasks created: {len(export_tasks)}")
print(f"{'='*70}")


# =========================
# 7. MONITOR EXPORT PROGRESS
# =========================

# Option 1: Monitor all tasks (will refresh every 60 seconds)
# Uncomment to use:
# monitor_export_tasks(export_tasks)

# Option 2: Check current status once
print("\nCURRENT TASK STATUS:")
print(f"{'='*70}\n")

states = {}
for i, task in enumerate(export_tasks):
    status = task.status()
    state = status['state']
    states[state] = states.get(state, 0) + 1

    if state == 'FAILED':
        print(f"Task {i+1}: {state} - {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Task {i+1}: {state}")

print(f"\n{'='*70}")
print("SUMMARY:")
for state, count in states.items():
    print(f"  {state}: {count}")
print(f"{'='*70}\n")

print("TIP: Run this cell again to check updated status")
print("Or visit: https://code.earthengine.google.com/tasks")


EXPORTING COMPOSITE AS TFRECORD
Number of tiles: 6
Patch size: 256x256 pixels
Scale: 10m
Output folder: tfrecord
Bands: ['0_VH', '1_VH', '2_VH', '3_VH', '4_VH', '5_VH', '6_VH', '7_VH', '8_VH', '9_VH', '10_VH', '11_VH', '12_VH', '13_VH', '14_VH', '15_VH', '16_VH', '17_VH']

[Tile 1/6] Preparing export...
   ✓ Task started: S1_composite_dry2025_tile_001
   Status: READY

[Tile 2/6] Preparing export...
   ✓ Task started: S1_composite_dry2025_tile_002
   Status: READY

[Tile 3/6] Preparing export...
   ✓ Task started: S1_composite_dry2025_tile_003
   Status: READY

[Tile 4/6] Preparing export...
   ✓ Task started: S1_composite_dry2025_tile_004
   Status: READY

[Tile 5/6] Preparing export...
   ✓ Task started: S1_composite_dry2025_tile_005
   Status: READY

[Tile 6/6] Preparing export...
   ✓ Task started: S1_composite_dry2025_tile_006
   Status: READY

✓ All 6 export tasks started successfully!

MONITORING:
  - Check task status in the next cell
  - Files will be saved to Google Drive
  

In [ ]:
gpkg_dir = '/content/drive/MyDrive/AGRI/Planting_Method/data'
output_dir = '/content/drive/MyDrive/AGRI/Planting_Method'

In [ ]:
bandnames = list(composite_mask.bandNames().getInfo())
bandnames

['0_VH',
 '1_VH',
 '2_VH',
 '3_VH',
 '4_VH',
 '5_VH',
 '6_VH',
 '7_VH',
 '8_VH',
 '9_VH',
 '10_VH',
 '11_VH',
 '12_VH',
 '13_VH',
 '14_VH',
 '15_VH',
 '16_VH',
 '17_VH']

In [ ]:
sorted_combined_images.toBands()

In [ ]:
from ee.ee_string import String
import os
import time
import pandas as pd
import geopandas as gpd
import ee

# =============================
# FAST SENTINEL EXTRACTION - KEEP ALL POINTS (CSV VERSION)
# =============================

def extract_reflectance_batch(image, points_fc, bands, scale=10):
    """
    Extract reflectance for multiple points in one server-side operation
    MUCH faster than individual point queries

    Args:
        image: ee.Image (Sentinel composite)
        points_fc: ee.FeatureCollection (points with properties)
        bands: List of band names
        scale: Resolution in meters

    Returns:
        List of dictionaries with reflectance values
    """

    # Sample all points at once (SERVER-SIDE)
    sampled = image.select(bands).sampleRegions(
        collection=points_fc,
        scale=scale,
        geometries=False,  # Don't return geometries (faster)
        tileScale=4  # Use for large areas
    )

    # Get results as list
    results = sampled.getInfo()['features']

    # Extract properties
    extracted_data = []
    for feature in results:
        props = feature['properties']
        extracted_data.append(props)

    return extracted_data


def csv_to_ee_featurecollection(df):
    """
    Convert DataFrame to Earth Engine FeatureCollection efficiently
    """
    features = []

    for idx, row in df.iterrows():
        # Create point geometry
        point = ee.Geometry.Point([row['gps_lon'], row['gps_lat']])

        # Create feature with properties
        feature = ee.Feature(point, {
            'id': int(idx),
            'gps_lat': float(row['gps_lat']),
            'gps_lon': float(row['gps_lon']),
            'crop_est_m': str(row['crop_est_m']),
            'crop_estab_d': str(row['crop_est_d'])
        })
        features.append(feature)

    return ee.FeatureCollection(features)


def process_csv_fast(file_path, sentinel_composite, bands, output_dir, chunk_size=2000):
    """
    Process CSV file with optimized batching
    KEEPS ALL POINTS - missing values will be interpolated later

    Args:
        file_path: Path to CSV file
        sentinel_composite: ee.Image
        bands: List of bands to extract
        output_dir: Output directory
        chunk_size: Number of points per batch (adjust based on memory)

    Returns:
        DataFrame with ALL results (including points with missing values)
    """

    file_name = os.path.basename(file_path)
    print(f"\n{'='*60}")
    print(f"Processing: {file_name}")
    print(f"{'='*60}")

    # Read CSV
    df = pd.read_csv(file_path)
    total_points = len(df)
    print(f"Total points: {total_points}")

    # Process in chunks to avoid memory issues
    all_results = []

    for i in range(0, total_points, chunk_size):
        chunk_end = min(i + chunk_size, total_points)
        chunk_df = df.iloc[i:chunk_end]

        print(f"\nProcessing chunk {i//chunk_size + 1}/{(total_points-1)//chunk_size + 1} "
              f"(points {i} to {chunk_end})")

        try:
            # Convert to FeatureCollection
            points_fc = csv_to_ee_featurecollection(chunk_df)

            # Extract reflectance (single server call for entire chunk!)
            start_time = time.time()
            results = extract_reflectance_batch(
                sentinel_composite,
                points_fc,
                bands,
                scale=10
            )
            elapsed = time.time() - start_time

            print(f"  ✓ Extracted {len(results)} points in {elapsed:.2f}s "
                  f"({len(results)/elapsed:.1f} points/sec)")

            all_results.extend(results)

        except Exception as e:
            print(f"  ✗ Error in chunk: {e}")
            continue

    # Convert to DataFrame - this only has points EE returned
    if all_results:
        results_df = pd.DataFrame(all_results)
    else:
        results_df = pd.DataFrame()

    # CRITICAL: Merge back with original CSV to keep ALL points
    # Start with the original data
    output_df = df.copy()
    output_df['id'] = output_df.index  # Ensure ID column matches

    if len(results_df) > 0:
        # Merge the band data - this will keep all original rows
        # and fill with NaN where EE didn't return data
        for band in bands:
            if band in results_df.columns:
                # Create a mapping of id -> band_value
                band_map = results_df.set_index('id')[band]
                # Map to original dataframe
                output_df[band] = output_df['id'].map(band_map)
    else:
        # No results from EE - create columns with all NaN
        for band in bands:
            output_df[band] = None

    # Add source file
    output_df['source_file'] = file_name

    # Reorder columns
    cols = ['id', 'gps_lat', 'gps_lon', 'crop_est_m', 'crop_est_d'] + bands + ['source_file']
    output_df = output_df[[c for c in cols if c in output_df.columns]]

    # COUNT missing values
    points_returned = len(results_df)
    points_with_data = results_df[bands].notna().all(axis=1).sum() if len(results_df) > 0 else 0
    points_missing = total_points - points_returned

    print(f"\n{'='*60}")
    print(f"Results Summary:")
    print(f"  Total points in CSV: {total_points}")
    print(f"  Points returned by EE: {points_returned}")
    print(f"  Points with complete data: {points_with_data}")
    print(f"  Points with NO data (will be NaN): {points_missing}")
    print(f"  (Missing values will be interpolated later)")
    print(f"{'='*60}")

    return output_df  # Return ALL points from original CSV


def process_all_csv_fast(csv_dir, sentinel_composite, bands, output_dir,
                         chunk_size=1000, parallel=False):
    """
    Process all CSV files in directory
    KEEPS ALL POINTS - no filtering of missing values

    Args:
        csv_dir: Directory containing CSV files
        sentinel_composite: ee.Image
        bands: List of bands
        output_dir: Output directory
        chunk_size: Points per batch
        parallel: Use parallel processing for multiple files (experimental)
    """

    os.makedirs(output_dir, exist_ok=True)

    # Find all CSV files
    csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

    print(f"\n{'='*70}")
    print(f"FAST REFLECTANCE EXTRACTION (KEEPING ALL POINTS)")
    print(f"{'='*70}")
    print(f"Found {len(csv_files)} CSV files")
    print(f"Bands: {bands}")
    print(f"Chunk size: {chunk_size} points/batch")
    print(f"{'='*70}")

    results_summary = []

    for file in csv_files:
        file_path = os.path.join(csv_dir, file)

        # Process file
        df = process_csv_fast(
            file_path,
            sentinel_composite,
            bands,
            output_dir,
            chunk_size
        )

        if df is not None and len(df) > 0:
            # Save to CSV (with ALL points, including missing values)
            out_csv = os.path.join(output_dir, f"reflectance_{file.replace('.csv', '')}.csv")
            df.to_csv(out_csv, index=False)
            print(f"\n✅ Saved: {out_csv}")

            # Count missing values for summary
            points_with_complete_data = df[bands].notna().all(axis=1).sum()
            points_with_any_missing = df[bands].isna().any(axis=1).sum()

            results_summary.append({
                'file': file,
                'total_points': len(df),
                'complete_points': points_with_complete_data,
                'points_with_missing': points_with_any_missing,
                'output': out_csv
            })
        else:
            print(f"\n❌ Could not process {file}")
            results_summary.append({
                'file': file,
                'total_points': 0,
                'complete_points': 0,
                'points_with_missing': 0,
                'output': None
            })

    # Print final summary
    print(f"\n{'='*70}")
    print(f"PROCESSING COMPLETE")
    print(f"{'='*70}")

    summary_df = pd.DataFrame(results_summary)
    print(summary_df.to_string(index=False))

    total_extracted = summary_df['total_points'].sum()
    total_complete = summary_df['complete_points'].sum()
    total_missing = summary_df['points_with_missing'].sum()

    print(f"\nTotal extracted: {total_extracted} points")
    print(f"  Complete data: {total_complete} ({total_complete/total_extracted*100:.1f}%)")
    print(f"  With missing values: {total_missing} ({total_missing/total_extracted*100:.1f}%)")
    print(f"{'='*70}\n")

    return summary_df


# =============================
# METHOD 2: BATCH EXPORT TO GOOGLE DRIVE - KEEPS ALL POINTS (CSV VERSION)
# =============================

def export_multiple_csv_to_drive(csv_dir, sentinel_composite, bands, scale=10):
    """
    Export reflectance for all CSV files to Google Drive in batch
    Fastest method - processes everything server-side
    ALL POINTS are exported (including those with missing values)

    Args:
        csv_dir: Directory with CSV files
        sentinel_composite: ee.Image
        bands: List of bands to extract
        scale: Resolution in meters

    Returns:
        List of export tasks
    """

    print(f"\n{'='*70}")
    print("BATCH EXPORT TO GOOGLE DRIVE (KEEPING ALL POINTS)")
    print(f"{'='*70}")

    # Find all CSV files
    csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
    print(f"Found {len(csv_files)} CSV files")
    print(f"Bands to extract: {bands}")
    print("Note: Points with missing values will be included for later interpolation")

    tasks = []

    for file in csv_files:
        file_path = os.path.join(csv_dir, file)
        file_name = file.replace('.csv', '')

        print(f"\n{'─'*60}")
        print(f"Processing: {file}")

        try:
            # Read CSV
            df = pd.read_csv(file_path)
            n_points = len(df)
            print(f"  Points: {n_points}")

            # Convert to FeatureCollection
            print(f"  Converting to Earth Engine format...")
            points_fc = csv_to_ee_featurecollection(df)

            # Sample regions - ALL points will be included
            print(f"  Preparing export...")

            # Buffer the points collection by 10 meters
            points_fc_buffered = points_fc.map(lambda feature: feature.buffer(10))


            sampled = sentinel_composite.select(bands).sampleRegions(
                collection=points_fc_buffered,
                scale=10,
                geometries=False,
                tileScale=8  # Higher for large areas
            )

            # Create export task
            export_name = f'reflectance_{file_name}'
            task = ee.batch.Export.table.toDrive(
                collection=sampled,
                description=export_name,
                fileNamePrefix=export_name,
                fileFormat='CSV',
                folder='planting_method',
                selectors=['id', 'gps_lat', 'gps_lon', 'crop_est_m', 'crop_est_d'] + bands
            )

            # Start task
            task.start()
            tasks.append({
                'file': file,
                'task': task,
                'export_name': export_name,
                'points': n_points
            })

            print(f"  ✓ Export task started: {export_name}")

        except Exception as e:
            print(f"  ✗ Error: {e}")
            continue

    # Print summary
    print(f"\n{'='*70}")
    print(f"EXPORT TASKS STARTED")
    print(f"{'='*70}")
    print(f"Total files: {len(csv_files)}")
    print(f"Successful tasks: {len(tasks)}")

    print(f"\n{'Task Summary':}")
    print(f"{'─'*70}")
    for t in tasks:
        print(f"  {t['file']:30s} → {t['export_name']:40s} ({t['points']:,} points)")

    print(f"\n{'Next Steps':}")
    print(f"{'─'*70}")
    print("1. Monitor tasks at: https://code.earthengine.google.com/tasks")
    print("2. Wait for all tasks to complete (usually 5-30 minutes)")
    print("3. Download CSV files from Google Drive")
    print("4. Files will be in Drive root or 'planting_method' folder")
    print("5. Interpolate missing values in post-processing")
    print(f"{'='*70}\n")

    return tasks


def monitor_export_tasks(tasks, check_interval=60):
    """
    Monitor export task progress

    Args:
        tasks: List of task dictionaries from export_multiple_csv_to_drive
        check_interval: Seconds between status checks
    """

    print(f"\n{'='*70}")
    print("MONITORING EXPORT TASKS")
    print(f"{'='*70}")
    print(f"Checking every {check_interval} seconds...")
    print("Press Ctrl+C to stop monitoring\n")

    try:
        while True:
            all_done = True
            status_summary = {'COMPLETED': 0, 'RUNNING': 0, 'FAILED': 0, 'PENDING': 0}

            print(f"\n{'Status Update':}")
            print(f"{'─'*70}")

            for t in tasks:
                task = t['task']
                status = task.status()
                state = status['state']

                status_summary[state] = status_summary.get(state, 0) + 1

                if state != 'COMPLETED':
                    all_done = False

                # Status indicators
                if state == 'COMPLETED':
                    icon = '✓'
                elif state == 'RUNNING':
                    icon = '⟳'
                elif state == 'FAILED':
                    icon = '✗'
                else:
                    icon = '○'

                print(f"  {icon} {t['export_name']:40s} | {state:10s}")

            print(f"\n{'Summary':}")
            print(f"{'─'*70}")
            for state, count in status_summary.items():
                if count > 0:
                    print(f"  {state:10s}: {count}")

            if all_done:
                print(f"\n{'='*70}")
                print("✓ ALL TASKS COMPLETED!")
                print(f"{'='*70}")
                print("Download your CSV files from Google Drive")
                break

            time.sleep(check_interval)

    except KeyboardInterrupt:
        print("\n\nMonitoring stopped. Tasks continue running on server.")
        print("Check status at: https://code.earthengine.google.com/tasks")


def download_and_organize_results(drive_dir, output_dir, tasks):
    """
    After tasks complete, organize downloaded files

    Args:
        drive_dir: Directory where you downloaded Drive files
        output_dir: Where to organize final CSVs
        tasks: Task list from export_multiple_csv_to_drive
    """

    os.makedirs(output_dir, exist_ok=True)

    print(f"\n{'='*70}")
    print("ORGANIZING DOWNLOADED FILES")
    print(f"{'='*70}")

    organized = 0

    for t in tasks:
        export_name = t['export_name']
        csv_name = f"{export_name}.csv"

        # Look for file in drive_dir
        source = os.path.join(drive_dir, csv_name)
        dest = os.path.join(output_dir, csv_name)

        if os.path.exists(source):
            # Copy to output directory
            import shutil
            shutil.copy2(source, dest)

            # Load and show summary
            df = pd.read_csv(dest)

            # Check for missing values
            bands = [col for col in df.columns if col not in ['id', 'gps_lat', 'gps_lon', 'crop_est_m', 'crop_est_d']]
            missing = df[bands].isna().any(axis=1).sum() if bands else 0

            print(f"✓ {csv_name:40s} | {len(df):,} points ({missing} with missing values)")
            organized += 1
        else:
            print(f"✗ {csv_name:40s} | Not found in {drive_dir}")

    print(f"\n{'='*70}")
    print(f"Organized {organized}/{len(tasks)} files")
    print(f"Output directory: {output_dir}")
    print(f"{'='*70}\n")


# =============================
# CONVENIENCE FUNCTION: ALL-IN-ONE
# =============================

def process_multiple_csv_drive_export(csv_dir, sentinel_composite, bands,
                                     monitor=True, check_interval=60):
    """
    All-in-one function: Export all CSV files and optionally monitor

    Args:
        csv_dir: Directory with CSV files
        sentinel_composite: ee.Image
        bands: List of bands
        monitor: Whether to monitor task progress
        check_interval: Seconds between monitoring checks

    Returns:
        List of export tasks
    """

    # Start all exports
    tasks = export_multiple_csv_to_drive(
        csv_dir=csv_dir,
        sentinel_composite=sentinel_composite,
        bands=bands,
        scale=10
    )

    # Monitor if requested
    if monitor and len(tasks) > 0:
        print("\nStarting task monitoring in 10 seconds...")
        time.sleep(10)  # Give tasks time to start
        monitor_export_tasks(tasks, check_interval)

    return tasks


# =============================
# CSV VALIDATION AND PREPARATION
# =============================

def validate_csv_for_ee(file_path):
    """
    Validate CSV file for Earth Engine compatibility
    """
    print(f"Validating: {file_path}")

    try:
        df = pd.read_csv(file_path)

        # Check required columns
        required_cols = ['gps_lat', 'gps_lon']
        missing_cols = [col for col in required_cols if col not in df.columns]

        if missing_cols:
            print(f"  ✗ Missing required columns: {missing_cols}")
            return False

        # Check data types
        try:
            df['gps_lat'] = pd.to_numeric(df['gps_lat'])
            df['gps_lon'] = pd.to_numeric(df['gps_lon'])
        except:
            print("  ✗ Coordinates must be numeric")
            return False

        # Check coordinate ranges
        if (df['gps_lat'].min() < -90 or df['gps_lat'].max() > 90 or
            df['gps_lon'].min() < -180 or df['gps_lon'].max() > 180):
            print("  ✗ Coordinate values out of range")
            return False

        # Check for null values in coordinates
        if df[['gps_lat', 'gps_lon']].isna().any().any():
            print("  ✗ Null values found in coordinates")
            return False

        print(f"  ✓ Valid CSV: {len(df)} points")
        return True

    except Exception as e:
        print(f"  ✗ Error reading CSV: {e}")
        return False


def prepare_csv_directory(csv_dir):
    """
    Validate all CSV files in directory
    """
    csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
    valid_files = []

    print(f"\nValidating {len(csv_files)} CSV files...")

    for file in csv_files:
        file_path = os.path.join(csv_dir, file)
        if validate_csv_for_ee(file_path):
            valid_files.append(file)

    print(f"\n✓ {len(valid_files)}/{len(csv_files)} files are valid for Earth Engine")
    return valid_files


# =============================
# EXAMPLE USAGE
# =============================

if __name__ == "__main__":

    # Example usage
    csv_directory = "/content/drive/MyDrive/AGRI/Planting_Method/csv"
    output_directory = "/content/drive/MyDrive/AGRI/Planting_Method"

    # Validate CSV files first
    valid_files = prepare_csv_directory(csv_directory)

    if valid_files:
        # Start the export process
        tasks = export_multiple_csv_to_drive(
            csv_dir=csv_directory,
            sentinel_composite=composite_mask,  # Replace with your ee.Image
            bands=bandnames,  # Replace with your bands
            scale=10
        )

        if tasks:
            print(f"\n{'✅ EXPORTS STARTED ':*^70}")
            print(f"Monitoring tasks at: https://code.earthengine.google.com/tasks")

            # Optional: Monitor progress
            monitor_export_tasks(tasks, check_interval=30)
        else:
            print("❌ No tasks started. Check your CSV directory path.")

    print("\n" + "="*70)
    print("OPTIMIZATION GUIDE")
    print("="*70)
    print("Original method: ~1-2 points/sec")
    print("Batch method:    ~50-200 points/sec (10-100x faster)")
    print("Export method:   ~1000+ points/sec (500x faster)")
    print("\nFor 1.4 MB (~10k points):")
    print("  Original: 1-3 hours")
    print("  Batch:    1-3 minutes ⚡")
    print("  Export:   5-15 seconds ⚡⚡⚡")
    print("\nNote: All points are kept, including those with missing values")
    print("      Missing values should be interpolated in post-processing")
    print("="*70)


Validating 2 CSV files...
Validating: /content/drive/MyDrive/AGRI/Planting_Method/csv/2024_Dry-Season.csv
  ✓ Valid CSV: 1023 points
Validating: /content/drive/MyDrive/AGRI/Planting_Method/csv/2025_Dry-Season.csv
  ✓ Valid CSV: 764 points

✓ 2/2 files are valid for Earth Engine

BATCH EXPORT TO GOOGLE DRIVE (KEEPING ALL POINTS)
Found 2 CSV files
Bands to extract: ['0_VH', '1_VH', '2_VH', '3_VH', '4_VH', '5_VH', '6_VH', '7_VH', '8_VH', '9_VH', '10_VH', '11_VH', '12_VH', '13_VH', '14_VH', '15_VH', '16_VH', '17_VH']
Note: Points with missing values will be included for later interpolation

────────────────────────────────────────────────────────────
Processing: 2024_Dry-Season.csv
  Points: 1023
  Converting to Earth Engine format...
  Preparing export...
  ✓ Export task started: reflectance_2024_Dry-Season

────────────────────────────────────────────────────────────
Processing: 2025_Dry-Season.csv
  Points: 764
  Converting to Earth Engine format...
  Preparing export...
  ✓ Export tas